In [1]:
import numpy as np
import pandas as pd
import os
import json
import collections
import re
from pathlib import Path
import seaborn as sns

In [2]:
import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    "font.serif": [],  
    'text.usetex': True,
    'pgf.rcfonts': False,
    'font.size': 8
})
import matplotlib.pyplot as plt


In [3]:
(mean, err) = pd.read_pickle("~/Programacio/cnn-limits/figures/cntk14_cv.pkl.gz")
mean = mean.drop(10, axis="columns")
err = err.drop(10, axis="columns")


In [4]:
sns.set_style("whitegrid")
sns.set_context("paper")

log_lengthscales = np.array([-2, *np.linspace(-1.5, 4.5, 25), 5])
lengthscales = 10**log_lengthscales

fig, axes = plt.subplots(1, 1, figsize=(5.50107, 3))
ax = axes

artists = []
labels = []

colors = plt.get_cmap('plasma')(np.linspace(0., 0.8, len(mean.columns)))
for column, color in zip(mean.columns[::-1], colors):
    x_axis = log_lengthscales
    y_axis = mean[column].values
    y_err = err[column]
    
    line, = ax.plot(x_axis, y_axis, color=color)
    labels.append(f"$N={column}$")
    artists.append(line)
    if column == 10240:
        
        indep_line, = ax.plot([x_axis[0]], [y_axis[0]], color=color, marker='<')
        meanp_line, = ax.plot([x_axis[-1]], [y_axis[-1]], color=color, marker='>')

        idx = mean.idxmax()[column]
        maxcv_line, =ax.plot([x_axis[idx]], [y_axis[idx]], color=color, marker='*')
    else:
        ax.plot(x_axis, y_axis+y_err, color=color, linewidth=0.5, ls='-', alpha=0.5)
        ax.plot(x_axis, y_axis-y_err, color=color, linewidth=0.5, ls='-', alpha=0.5)
        
        ax.plot([x_axis[0]], [y_axis[0]], color=color, marker='<')
        ax.plot([x_axis[-1]], [y_axis[-1]], color=color, marker='>')
    
        idx = mean.idxmax()[column]
        ax.plot([x_axis[idx]], [y_axis[idx]], color=color, marker='*')
    #if column == 10240:
    #    y_upper = mean.loc[idx, column] - 0.01
    #else:
    #    y_upper = mean.loc[idx, column] - err.loc[idx, column]-0.01
    #x_bar = x_axis[idx]
    #ax.plot([x_bar, x_bar], [y_upper, 0.1], color=color, ls='--', linewidth=1)
    
#empty_artist, = ax.plot([], [], linewidth=0.0)
ax.set_ylabel("4-fold cross-validation accuracy")
ax.set_xlabel("$log_{10}($lengthscale$)$ of Matérn-3/2 kernel")
ax.legend((*artists, indep_line, meanp_line, maxcv_line),
          (*labels, "indep. $W$", "mean pool", "max. CV acc."),
          frameon=False, ncol=1 , loc='center left', bbox_to_anchor=(1, 0.5, 0, 0))

fig.tight_layout()
plt.savefig("../neurips2020/fig/cntk14.pgf")